In [1]:
import mod1

In [2]:
a = mod1.excel()
a.kospi_stock_price()

900140 엘브이엠씨홀딩스
              Code      Name  Open  High   Low   Volume  Close
Date                                                          
2019-01-02  900140  엘브이엠씨홀딩스  2340  2390  2320    89587   2355
2019-01-03  900140  엘브이엠씨홀딩스  2340  2385  2320    65074   2335
2019-01-04  900140  엘브이엠씨홀딩스  2295  2355  2250    79248   2355
2019-01-07  900140  엘브이엠씨홀딩스  2360  2500  2355   156781   2475
2019-01-08  900140  엘브이엠씨홀딩스  2480  2645  2450   248480   2465
2019-01-09  900140  엘브이엠씨홀딩스  2580  2850  2550  2636568   2645
2019-01-10  900140  엘브이엠씨홀딩스  2640  2685  2600   298585   2625
2019-01-11  900140  엘브이엠씨홀딩스  2625  2895  2610  1081334   2735
2019-01-14  900140  엘브이엠씨홀딩스  2785  2785  2680   179188   2710
2019-01-15  900140  엘브이엠씨홀딩스  2710  2755  2670   188285   2700
2019-01-16  900140  엘브이엠씨홀딩스  2690  2720  2640   189236   2655
2019-01-17  900140  엘브이엠씨홀딩스  2660  2700  2630   114812   2690
2019-01-18  900140  엘브이엠씨홀딩스  2690  2785  2675   160987   2725
2019-01-21  900140  엘브이엠씨홀딩스  2715  281

In [6]:
a.kosdaq_stock_price('2019-1-1', '2019-1-10')

126640 화신정공
              Code  Name  Open  High   Low  Volume  Close
Date                                                     
2019-01-02  126640  화신정공  1165  1170  1140  115024   1170
2019-01-03  126640  화신정공  1170  1175  1145  181180   1150
2019-01-04  126640  화신정공  1145  1150  1120  107070   1150
2019-01-07  126640  화신정공  1150  1215  1135  280026   1145
2019-01-08  126640  화신정공  1150  1165  1145  115216   1155
2019-01-09  126640  화신정공  1165  1180  1150  164918   1175
2019-01-10  126640  화신정공  1175  1210  1170  313198   1210
